In [0]:
import numpy as np
from keras import backend
import tensorflow as tf
import keras as ks
import os
import cv2
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout,GlobalAveragePooling2D,Input,Reshape,Lambda,GRU,LSTM,TimeDistributed

In [0]:
# Download the Dataset
# !wget https://ucdcfedc1081bafd8a48fc17451c.dl.dropboxusercontent.com/cd/0/get/ArkPU0wDsLhki-AfQmctuqCTQuXTXunaHMvsEQK5qY_hQ84gPhHLLQgKWOCCthZqbPbf4iyk_q9oHy3GhFu9Ur3A7l5EW_T5m8s6Y4Ijoq_hvA/file?_download_id=5557862267060294899674724915065891683584576254567948294615437415&_notify_domain=www.dropbox.com&dl=1

In [0]:
# UNZIP THE DATA
# !tar -xvf data

In [0]:
# Getting the data directories
frames_basket=os.listdir('./data/train/action0')
frames_diving=os.listdir('./data/train/action5')
val_b=os.listdir('./data/val/action0')
val_d=os.listdir('./data/val/action5')
test_b=os.listdir('./data/test/action0')
test_d=os.listdir('./data/test/action5')

In [0]:
#loading the meta-data in memory 
def data_prep(frames):
  data={}
  for i in frames:
    scene=i.split("_")[2]
    if scene not in data.keys():  
      data[scene]={}
    clip,fr=i.split("_")[-1].split(".")[0:2]
    if clip not in data[scene].keys():
      data[scene][clip]=[]
    data[scene][clip].append(int(fr))

  for key in data.keys():
    for clip in data[key].keys():
      data[key][clip]=sorted(data[key][clip])

  return data

data_basketball=data_prep(frames_basket)
data_diving=data_prep(frames_diving)
val_basketball=data_prep(val_b)
val_diving=data_prep(val_d)
test_basketball=data_prep(test_b)
test_diving=data_prep(test_d)

In [0]:
len(data_diving.keys()),len(data_diving['g02'].keys()),21*7

(21, 7, 147)

## processed_data directory structure

processed_data                     
|  
|-----Basketball  
|--------|  
|--------|---clipone  
|--------|---cliptwo  
|--------|---clipthree ...  
|  
|  
|------Diving  
|--------|  
|--------|---clipone  
|--------|---cliptwo  
|--------|---clipthree ...  
|  

In [0]:
#deleting the data
# !rm -rf /content/processed_data_train/Basketball
# !rm -rf /content/processed_data_train/Diving
# !rm -rf /content/processed_val_train/Basketball
# !rm -rf /content/processed_val_train/Diving

In [0]:
#Processing the data 
"""Applying Background subtractor and storing the images in the above directory format"""
def process_data(data,category,path,new=0,prim_path="./processed_data_train/"):
    if not os.path.exists(prim_path+category+"/"):
        os.makedirs(prim_path+category+"/")
    num=0
    if new == 1:
        num=len(os.listdir(prim_path+category+"/"))
    for key in data.keys():
        for clip in data[key].keys():
            fgbg = cv2.createBackgroundSubtractorMOG2(history = 50,varThreshold = 200,) 
            if not os.path.exists(prim_path+category+"/clip_"+str(num)):
                os.mkdir(prim_path+category+"/clip_"+str(num))
            frame_num=0
            for frame in data[key][clip]:
                img=cv2.resize(cv2.imread(path+"v_"+category+"_"+key+"_"+clip+"."+"0"*(4-len(str(frame)))+str(frame)+".jpg"),(240,320))               
                cv2.imwrite(prim_path+category+"/clip_"+str(num)+"/"+str(frame_num)+".jpg",fgbg.apply(img),) 
                frame_num+=1
            num+=1

In [0]:
#Processing Data
# process_data(data_basketball,"Basketball","./data/train/action0/")
# process_data(data_diving,"Diving","./data/train/action5/")
# process_data(val_diving,"Diving","./data/val/action5/",0,"./processed_val_train/")
# process_data(val_basketball,"Basketball","./data/val/action0/",0,"./processed_val_train/")
# process_data(test_diving,"Diving","./data/test/action5/",1,"./processed_val_train/")
# process_data(test_basketball,"Basketball","./data/test/action0/",1,"./processed_val_train/")

In [0]:
#appending the processed images in a time series

def prepare_clip(path,time_frames=10,im_shape=(96,96)):
    clip=[]
    for i in range(time_frames):
        clip.append(cv2.resize(cv2.imread(path+"/"+str(i+1)+".jpg"),im_shape)/255.)
    return clip

def prepare_data(path_b="./processed_data_train/Basketball/",
                 path_d="./processed_data_train/Diving/",batch_size=32,im_shape=(96,96),time_frames=10):
    
    list_b=os.listdir(path_b)
    list_d=os.listdir(path_d)
    np.random.shuffle(list_b)
    np.random.shuffle(list_d)
    b,d=0,0
    
    while True:
        x=[]
        y=[]
        if(b>=len(list_b)):
            np.random.shuffle(list_b)
            b=0
        if(d>=len(list_d)):
            np.random.shuffle(list_d)
            d=0
        for i in range(batch_size):
            if i%2 == 0:

                x.append(prepare_clip(path_b+list_b[b],time_frames=time_frames,im_shape=im_shape))
                y.append(0)
                b+=1
                if(b>=len(list_b)):
                    np.random.shuffle(list_b)
                    b=0
            else:
                x.append(prepare_clip(path_d+list_d[d],time_frames=time_frames,im_shape=im_shape))
                y.append(1)
                d+=1
                if(d>=len(list_d)):
                    np.random.shuffle(list_d)
                    d=0
        yield (np.array(x),np.array(y))


In [0]:
#CNN-LSTMS Time series model 
def create_model(input_shape):

  L1=Input(shape=input_shape)
  L3=TimeDistributed(Conv2D(filters=128,kernel_size=(4,4),strides=1,padding='SAME',activation="relu"))(L1)
  L4=TimeDistributed(Conv2D(filters=128,kernel_size=(4,4),strides=1,padding='SAME',activation="relu"))(L3)
  L5=TimeDistributed(MaxPool2D(strides=2))(L4)
  L6=TimeDistributed(Conv2D(filters=128,kernel_size=(4,4),strides=1,padding='SAME',activation="relu"))(L5)
  L7=TimeDistributed(Conv2D(filters=128,kernel_size=(4,4),strides=1,padding='SAME',activation="relu"))(L6)
  L8=TimeDistributed(MaxPool2D(strides=2))(L7)
  L9=TimeDistributed(Conv2D(filters=128,kernel_size=(3,3),strides=1,padding='SAME',activation="relu"))(L8)
  L10=TimeDistributed(Conv2D(filters=64,kernel_size=(3,3),strides=1,padding='SAME',activation="relu"))(L9)
  L11=TimeDistributed(MaxPool2D(strides=2))(L10)
  L12=TimeDistributed(Conv2D(filters=64,kernel_size=(3,3),strides=1,padding='SAME',activation="relu"))(L11)
  L13=TimeDistributed(GlobalAveragePooling2D())(L12)

  L15=GRU(128,return_sequences=True)(L13)
  L16=GRU(64,return_sequences=True)(L15)

  L17=Dense(128,activation="relu")(Dropout(0.25)(Flatten()(L16)))
  L18=Dense(64,activation="relu")(L17)
  L19=Dense(32,activation="relu")(L18)
  L19=Dropout(0.25)(L19)
  L20=Dense(1,activation="sigmoid")(L19)


  model=ks.models.Model(input=[L1],output=[L20])
  model.compile(optimizer=ks.optimizers.Adam(lr=0.0001),
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [0]:
total_data=(len(os.listdir("./processed_data_train/Basketball/"))+len(os.listdir("./processed_data_train/Diving/")))

# print(total_data)

image_shape=96
batch_size=32
time_frames=10

steps_per_epoch=total_data//batch_size.
# print(steps_per_epoch)

model=create_model(input_shape=(time_frames,image_shape,image_shape,3))
# gen=prepare_data(batch_size=batch_size,im_shape=(image_shape,image_shape),time_frames=time_frames)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`


In [0]:
total_data_val=(len(os.listdir("./processed_val_train/Basketball/"))+len(os.listdir("./processed_val_train/Diving/")))
total_data_val

63

In [0]:
#getting val_data
val_data=next(prepare_data(path_b="./processed_val_train/Basketball/",
                 path_d="./processed_val_train/Diving/",
                 batch_size=total_data_val,
                im_shape=(image_shape,image_shape),
                time_frames=time_frames))

In [0]:
#training
model.fit_generator(gen,steps_per_epoch=steps_per_epoch,epochs=5,validation_data=val_data,)

Epoch 1/5
6/6 [==============================] - 21s 4s/step - loss: 0.4272 - acc: 0.7969 - val_loss: 0.4821 - val_acc: 0.8254
Epoch 2/5
6/6 [==============================] - 18s 3s/step - loss: 0.5596 - acc: 0.7448 - val_loss: 0.7622 - val_acc: 0.6825
Epoch 3/5
6/6 [==============================] - 18s 3s/step - loss: 0.5248 - acc: 0.7344 - val_loss: 0.5110 - val_acc: 0.7778
Epoch 4/5
6/6 [==============================] - 18s 3s/step - loss: 0.4520 - acc: 0.7812 - val_loss: 0.5172 - val_acc: 0.7778
Epoch 5/5
6/6 [==============================] - 18s 3s/step - loss: 0.4726 - acc: 0.7656 - val_loss: 0.5206 - val_acc: 0.7778
